Final project

In [339]:
import numpy as np
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt

In [340]:
pd.set_option('display.max_columns', None) # to show all the columns in output window

### Cancer set

In [341]:
#load cancer dataset to be used in the target database
cancer = pd.read_excel('cancer.xlsx')

In [342]:
#Dropt the duplicate
can_drop = cancer.drop_duplicates(ignore_index=True)
can_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Patient Id                1000 non-null   object
 1   Age                       1000 non-null   int64 
 2   Gender                    1000 non-null   int64 
 3   Air Pollution             1000 non-null   int64 
 4   Alcohol use               1000 non-null   int64 
 5   Dust Allergy              1000 non-null   int64 
 6   OccuPational Hazards      1000 non-null   int64 
 7   Genetic Risk              1000 non-null   int64 
 8   chronic Lung Disease      1000 non-null   int64 
 9   Balanced Diet             1000 non-null   int64 
 10  Obesity                   1000 non-null   int64 
 11  Smoking                   1000 non-null   int64 
 12  Passive Smoker            1000 non-null   int64 
 13  Chest Pain                1000 non-null   int64 
 14  Coughing of Blood        

In [343]:
#Replace space with underscore in column names
cancer.columns = [i.replace(' ','_') for i in cancer.columns]
#Replace colume name with first letter capitalized
cancer.columns = cancer.columns.str.replace(r'(\w+)', lambda x: x.group().lower(),
                                    n=2, regex=True)
#Replace level column name
cancer.columns = [i.replace('level','cancer_level') for i in cancer.columns]
cancer.head(2)

,patient_id,age,gender,air_pollution,alcohol_use,dust_allergy,occupational_hazards,genetic_risk,chronic_lung_disease,balanced_diet,obesity,smoking,passive_smoker,chest_pain,coughing_of_blood,fatigue,weight_loss,shortness_of_breath,wheezing,swallowing_difficulty,clubbing_of_finger_nails,frequent_cold,dry_cough,snoring,cancer_level
0,P1,33,1,2,4,5,4,3,2,2,4,3,2,2,4,3,4,2,2,3,1,2,3,4,Low
1,P10,17,1,3,1,5,3,4,2,2,2,2,4,2,3,1,3,7,8,6,2,1,7,2,Medium


-"Patient_ID" is drop for protecting privacy
-"Smoking" and "passive smoking" and "obisity" are drop, since similar feature can be obtain from insurance dataset

In [344]:
#drop 'patient_id', "Smoking" and "passive smoking" and "obisity"
cancer_drop = cancer.drop(['patient_id','obesity', 'smoking', 'passive_smoker'], axis=1)
cancer_drop.head(2)

,age,gender,air_pollution,alcohol_use,dust_allergy,occupational_hazards,genetic_risk,chronic_lung_disease,balanced_diet,chest_pain,coughing_of_blood,fatigue,weight_loss,shortness_of_breath,wheezing,swallowing_difficulty,clubbing_of_finger_nails,frequent_cold,dry_cough,snoring,cancer_level
0,33,1,2,4,5,4,3,2,2,2,4,3,4,2,2,3,1,2,3,4,Low
1,17,1,3,1,5,3,4,2,2,2,3,1,3,7,8,6,2,1,7,2,Medium


### Insurance set

In [345]:
#load insurance dataset to be used in the target database
insurance = pd.read_csv('insurance.csv')
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [346]:
#check for smoker distribution
insurance['smoker'].value_counts()


no     1064
yes     274
Name: smoker, dtype: int64

In [347]:
#encode sex feature
encoding = {"male": 1, "female": 2}
# Apply the encoding to the "Gender" column
insurance["gender"] = insurance["sex"].map(encoding)

# Convert bmi to level of Low, Normal, and High considering 18.5-24.9 is a normal range
def bmi_category(bmi):
    if bmi < 18.5:
        return "low"
    elif bmi >= 18.5 and bmi <= 24.9:
        return "normal"
    else:
        return "high"
insurance["bmi_cat"] = insurance["bmi"].apply(bmi_category)

# Convert charges to charges level of 1:0- <5k, 2: 5K- <10K, 3: 10K- <15K, 4: >15K
def charge_level(charges):
    if charges < 5000:
        return 1
    elif charges >= 5000 and charges < 10000:
        return 2
    elif charges >= 10000 and charges < 15000:
        return 3
    else:
        return 4
insurance["charge_level"] = insurance["charges"].apply(charge_level)

insurance.head()

,age,sex,bmi,children,smoker,region,charges,gender,bmi_cat,charge_level
0,19,female,27.900,0,yes,southwest,16884.92400,2,high,4
1,18,male,33.770,1,no,southeast,1725.55230,1,high,1
2,28,male,33.000,3,no,southeast,4449.46200,1,high,1
3,33,male,22.705,0,no,northwest,21984.47061,1,normal,4
4,32,male,28.880,0,no,northwest,3866.85520,1,high,1


In [348]:
#Drop bmi and sex
in_drop = insurance.drop(columns=["sex", "bmi", "charges"])

In [349]:
#Dropt the duplicate
in_drop_test =in_drop.drop_duplicates(ignore_index=True)
in_drop_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1098 entries, 0 to 1097
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   age           1098 non-null   int64 
 1   children      1098 non-null   int64 
 2   smoker        1098 non-null   object
 3   region        1098 non-null   object
 4   gender        1098 non-null   int64 
 5   bmi_cat       1098 non-null   object
 6   charge_level  1098 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 60.2+ KB


### Merging datasets to normalize the data

In [350]:
#Merge to dataset 
merged_df = pd.merge(cancer_drop, in_drop, on=["age", "gender"])
merged_df.head(5)

,age,gender,air_pollution,alcohol_use,dust_allergy,occupational_hazards,genetic_risk,chronic_lung_disease,balanced_diet,chest_pain,coughing_of_blood,fatigue,weight_loss,shortness_of_breath,wheezing,swallowing_difficulty,clubbing_of_finger_nails,frequent_cold,dry_cough,snoring,cancer_level,children,smoker,region,bmi_cat,charge_level
0,33,1,2,4,5,4,3,2,2,2,4,3,4,2,2,3,1,2,3,4,Low,0,no,northwest,normal,4
1,33,1,2,4,5,4,3,2,2,2,4,3,4,2,2,3,1,2,3,4,Low,2,no,southeast,high,1
2,33,1,2,4,5,4,3,2,2,2,4,3,4,2,2,3,1,2,3,4,Low,0,no,northeast,high,3
3,33,1,2,4,5,4,3,2,2,2,4,3,4,2,2,3,1,2,3,4,Low,1,no,southeast,high,3
4,33,1,2,4,5,4,3,2,2,2,4,3,4,2,2,3,1,2,3,4,Low,5,no,southwest,high,2


In [351]:
merged_df.shape

(13457, 26)

In [352]:
duplicate_rows = merged_df[merged_df.duplicated()]
duplicate_rows.shape
# Print the duplicate rows
#print(duplicate_rows)

(11735, 26)

In [353]:
#Dropt the duplicate
merged_df_drop = merged_df.drop_duplicates(ignore_index=True)
merged_df_drop.shape

(1722, 26)

In [354]:
#Save the merged data into csv file
merged_df_drop.to_csv("cancer_insurance.csv", index=False)

#### Start Normalize data

In [355]:
df = pd.read_csv("cancer_insurance.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1722 entries, 0 to 1721
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   age                       1722 non-null   int64 
 1   gender                    1722 non-null   int64 
 2   air_pollution             1722 non-null   int64 
 3   alcohol_use               1722 non-null   int64 
 4   dust_allergy              1722 non-null   int64 
 5   occupational_hazards      1722 non-null   int64 
 6   genetic_risk              1722 non-null   int64 
 7   chronic_lung_disease      1722 non-null   int64 
 8   balanced_diet             1722 non-null   int64 
 9   chest_pain                1722 non-null   int64 
 10  coughing_of_blood         1722 non-null   int64 
 11  fatigue                   1722 non-null   int64 
 12  weight_loss               1722 non-null   int64 
 13  shortness_of_breath       1722 non-null   int64 
 14  wheezing                

In [397]:
#Generate cancer_id using groupby
df['cancer_id'] = (df.groupby(['chest_pain','coughing_of_blood','fatigue', 'weight_loss', 
                               "shortness_of_breath", "wheezing", "swallowing_difficulty", 
                               "clubbing_of_finger_nails", "frequent_cold", "dry_cough", "snoring", 'cancer_level'],
                                sort=False).ngroup().add(1))

#Generate patient_id 
df['patient_id'] = (df.groupby(['age','gender','children', 'bmi_cat', 'region', 'cancer_id'],
                                sort=False).ngroup().add(1))

#Generate insurance_id 
df['insurance_id'] = (df.groupby(['charge_level','patient_id'],
                                sort=False).ngroup().add(1))

#Generate behavior_id 
df['behavior_id'] = (df.groupby(['alcohol_use', 'smoker', 'balanced_diet', 'patient_id'],
                                sort=False).ngroup().add(1))
#Generate factor_id 
df['factor_id'] = (df.groupby(["air_pollution", "dust_allergy", "occupational_hazards", 
                "genetic_risk", "chronic_lung_disease", 'cancer_id'],
                                sort=False).ngroup().add(1))



In [398]:
df

,age,gender,air_pollution,alcohol_use,dust_allergy,occupational_hazards,genetic_risk,chronic_lung_disease,balanced_diet,chest_pain,coughing_of_blood,fatigue,weight_loss,shortness_of_breath,wheezing,swallowing_difficulty,clubbing_of_finger_nails,frequent_cold,dry_cough,snoring,cancer_level,children,smoker,region,bmi_cat,charge_level,cancer_id,patient_id,insurance_id,behavior_id,factor_id
0,33,1,2,4,5,4,3,2,2,2,4,3,4,2,2,3,1,2,3,4,Low,0,no,northwest,normal,4,1,1,1,1,1
1,33,1,2,4,5,4,3,2,2,2,4,3,4,2,2,3,1,2,3,4,Low,2,no,southeast,high,1,1,2,2,2,1
2,33,1,2,4,5,4,3,2,2,2,4,3,4,2,2,3,1,2,3,4,Low,0,no,northeast,high,3,1,3,3,3,1
3,33,1,2,4,5,4,3,2,2,2,4,3,4,2,2,3,1,2,3,4,Low,1,no,southeast,high,3,1,4,4,4,1
4,33,1,2,4,5,4,3,2,2,2,4,3,4,2,2,3,1,2,3,4,Low,5,no,southwest,high,2,1,5,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1717,29,1,6,7,7,7,7,6,7,7,7,2,7,6,7,6,7,2,3,1,High,3,no,northeast,normal,2,16,1555,1713,1672,16
1718,29,1,6,7,7,7,7,6,7,7,7,2,7,6,7,6,7,2,3,1,High,0,yes,northeast,normal,4,16,1556,1714,1673,16
1719,29,1,6,7,7,7,7,6,7,7,7,2,7,6,7,6,7,2,3,1,High,2,no,northwest,high,1,16,1551,1715,1667,16
1720,29,1,6,7,7,7,7,6,7,7,7,2,7,6,7,6,7,2,3,1,High,1,no,southeast,high,1,16,1557,1716,1674,16


In [399]:
#df['factor_id'].value_counts()

### Generating multiple data frames  matching relational tabel target from the merged dataframe

### Patient

In [403]:
#Create patient table
patient_df = df[['patient_id','age','gender', 'bmi_cat', 'children', 'region','cancer_id']]
#drop duplicate 
patient_df =  patient_df.drop_duplicates(ignore_index=True)
patient_df.head()

,patient_id,age,gender,bmi_cat,children,region,cancer_id
0,1,33,1,normal,0,northwest,1
1,2,33,1,high,2,southeast,1
2,3,33,1,high,0,northeast,1
3,4,33,1,high,1,southeast,1
4,5,33,1,high,5,southwest,1


In [404]:
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1558 entries, 0 to 1557
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   patient_id  1558 non-null   int64 
 1   age         1558 non-null   int64 
 2   gender      1558 non-null   int64 
 3   bmi_cat     1558 non-null   object
 4   children    1558 non-null   int64 
 5   region      1558 non-null   object
 6   cancer_id   1558 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 85.3+ KB


In [419]:
patient_df.to_csv("patient.csv", index=False)

### Insurance Info

In [406]:
#Create patient table
insurance_info_df = df[['insurance_id', 'charge_level', 'patient_id']]
#drop duplicate 
insurance_info_df = insurance_info_df.drop_duplicates(ignore_index=True)
insurance_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1717 entries, 0 to 1716
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   insurance_id  1717 non-null   int64
 1   charge_level  1717 non-null   int64
 2   patient_id    1717 non-null   int64
dtypes: int64(3)
memory usage: 40.4 KB


In [407]:
insurance_info_df.head()

,insurance_id,charge_level,patient_id
0,1,4,1
1,2,1,2
2,3,3,3
3,4,3,4
4,5,2,5


In [408]:
insurance_info_df.to_csv("insurance_info.csv", index=False)

### Factor

In [409]:
factor_df = df[["factor_id", "air_pollution", "dust_allergy", "occupational_hazards", 
                "genetic_risk", "chronic_lung_disease", 'cancer_id']]
factor_df = factor_df.drop_duplicates(ignore_index=True)
factor_df.head()

,factor_id,air_pollution,dust_allergy,occupational_hazards,genetic_risk,chronic_lung_disease,cancer_id
0,1,2,5,4,3,2,1
1,2,6,7,7,7,6,2
2,3,3,4,2,3,2,3
3,4,6,7,7,7,6,4
4,5,6,7,7,7,7,5


In [410]:
factor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   factor_id             50 non-null     int64
 1   air_pollution         50 non-null     int64
 2   dust_allergy          50 non-null     int64
 3   occupational_hazards  50 non-null     int64
 4   genetic_risk          50 non-null     int64
 5   chronic_lung_disease  50 non-null     int64
 6   cancer_id             50 non-null     int64
dtypes: int64(7)
memory usage: 2.9 KB


In [411]:
factor_df.to_csv("factor.csv", index=False)

### Symptom

In [412]:
symptom_df = df[['cancer_id','chest_pain','coughing_of_blood','fatigue', 'weight_loss', 
                 "shortness_of_breath", "wheezing", "swallowing_difficulty", 
                "clubbing_of_finger_nails", "frequent_cold", "dry_cough", "snoring", 'cancer_level']]
symptom_df  = symptom_df .drop_duplicates(ignore_index=True)
symptom_df.head()

,cancer_id,chest_pain,coughing_of_blood,fatigue,weight_loss,shortness_of_breath,wheezing,swallowing_difficulty,clubbing_of_finger_nails,frequent_cold,dry_cough,snoring,cancer_level
0,1,2,4,3,4,2,2,3,1,2,3,4,Low
1,2,7,7,4,4,5,6,5,5,4,6,5,High
2,3,3,3,4,5,6,5,5,4,6,5,4,Medium
3,4,7,7,9,6,5,7,2,4,3,1,4,High
4,5,7,7,8,5,7,6,7,8,7,6,2,High


In [413]:
symptom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cancer_id                 46 non-null     int64 
 1   chest_pain                46 non-null     int64 
 2   coughing_of_blood         46 non-null     int64 
 3   fatigue                   46 non-null     int64 
 4   weight_loss               46 non-null     int64 
 5   shortness_of_breath       46 non-null     int64 
 6   wheezing                  46 non-null     int64 
 7   swallowing_difficulty     46 non-null     int64 
 8   clubbing_of_finger_nails  46 non-null     int64 
 9   frequent_cold             46 non-null     int64 
 10  dry_cough                 46 non-null     int64 
 11  snoring                   46 non-null     int64 
 12  cancer_level              46 non-null     object
dtypes: int64(12), object(1)
memory usage: 4.8+ KB


In [414]:
symptom_df.to_csv("symptom.csv", index=False)

### Behavior

In [416]:
behavior_df = df[['behavior_id', 'alcohol_use', 'smoker', 'balanced_diet', 'patient_id']]
#Drop the duplicate
behavior_df  =  behavior_df .drop_duplicates(ignore_index=True)
behavior_df .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1675 entries, 0 to 1674
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   behavior_id    1675 non-null   int64 
 1   alcohol_use    1675 non-null   int64 
 2   smoker         1675 non-null   object
 3   balanced_diet  1675 non-null   int64 
 4   patient_id     1675 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 65.6+ KB


In [417]:
behavior_df.head()

,behavior_id,alcohol_use,smoker,balanced_diet,patient_id
0,1,4,no,2,1
1,2,4,no,2,2
2,3,4,no,2,3
3,4,4,no,2,4
4,5,4,no,2,5


In [420]:
behavior_df.to_csv("behavior.csv", index=False)